In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

c:\Users\prath\OneDrive\Desktop\UCSC\Research\Cloudnet-Research-Project-Repo\ml-models\VMFT_LAD\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from env import OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)
model = "gpt-3.5-turbo"

In [20]:
prompt_sample_logs = """
Following are some of the normal state logs, refer to them when deciding whether the given log template is a failure or normal.
Normal state log templates:
br0: port entered state,
device promiscuous mode,
delete user,
Failed for invalid user from port ssh2,
Could not find device,
waiting for device initialization: No such device",
uid= euid=,
Failed password for from port  ssh2,
Received disconnect from port :: disconnected by user,
tap1: Gained,
Lost carrier,
Stopped target,
Disconnected from user port ,
Stopped target Host and Network Name Lookups.,
Finished Exit the Session.,
Stopped User Manager for UID .,
FAILED LOGIN () on '/dev/tty1' FOR 'UNKNOWN', Authentication failure,
Stopping System Logging Service...
"""

def generate_prompt(log_line):
  prompt_text = ('Classify the Log line into faulty or normal.\n' +
  'Log line: "' + log_line + '"\n' +
  prompt_sample_logs + '\n'+
  'Command: Classify only whether the given log line as faulty or normal in one word \n' +
  'Result: ')
  messages = [{"role": "user", "content": prompt_text}]
  return messages

In [21]:
def infer_log_line(log_line: str) -> bool:
  response = client.chat.completions.create(
    model=model,
    messages=generate_prompt(log_line)
  )
  generated_text = response.choices[0].message.content
  return generated_text.lower().find("fault") >= 0

In [ ]:
infer_log_line('Using degraded feature set  for DNS server .')

In [ ]:
drain3_state_df = pd.read_csv('../../../../preprocessing/drain3_state.csv', header=0)
drain3_state_df

In [ ]:
out_df = drain3_state_df.copy()
labels = []

for index, row in tqdm(drain3_state_df.iterrows()):
    labels.append(infer_log_line(row["log_text"]))

out_df["label"] = labels

In [ ]:
out_df.to_csv("./GPT3_few_shot_out.csv", index=False)

In [3]:
import pandas as pd
drain3_state_df = pd.read_csv('../../../../preprocessing/drain3_state.csv', header=0)
out_df = drain3_state_df.copy()

labels = []
output = []
f = open("./gpt3_out_raw.csv", "r")
for line in f:
    result = line.strip().split(",")[1]
    output.append(result)
    labels.append(True if result.lower().strip() == "faulty" else False)

label_df = pd.DataFrame({"label":labels, "output":output})

In [4]:
out_df["label"] = label_df["label"]

out_df.to_csv("./GPT3_few_shot_out.csv", index=False)